In [1]:
import os 
import numpy as np
import matplotlib.pyplot as plt
import torch 
import torch.nn as nn
import torchvision
from torch.utils.data import DataLoader
from torchvision import models, datasets, transforms
import torch.optim as optim 
import torch.functional as F
from torch.utils.tensorboard import SummaryWriter

In [5]:
class Discriminator(nn.Module):
    
    def __init__(self, channels_img, features_d):
        super(Discriminator, self).__init__()
        # Did not use BatchNorm in the last layer of the generator and the first layer of the 
        # discriminator
        # Input: N x channels_img x 64 x 64
        self.discriminator = nn.Sequential(
            nn.Conv2d(channels_img, features_d, kernel_size = 4, stride = 2, padding = 1), #32x32
            nn.LeakyReLU(0.2),
            self._block(features_d, features_d*2, 4, 2, 1),# 16x16
            self._block(features_d*2, features_d*4, 4, 2, 1), #8x8
            self._block(features_d*4, features_d*8, 4, 2, 1), #4x4
            nn.Conv2d(features_d*8, 1, kernel_size = 4, stride = 2, padding = 0), #1x1(Prediction)
            nn.Sigmoid()
        )
    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels,kernel_size, stride, padding, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.LeakyReLU(0.2)
        )
    
    def forward(self,x):
        return self.discriminator(x)
       
                
class Generator(nn.Module):
    
    # Here channels_img is nothing but the inputs channels 
    # and features_g is nothing but the output channels
    def __init__(self, z_dim, channels_img, features_g):
        
        super(Generator, self).__init__()
        
        self.net = nn.Sequential(
            # Input: N x z_dim x 1 x 1
            self._block(z_dim, features_g*16, 4, 1, 0), # N x f_g*16(1024=64*64) x 4 x 4
            self._block(features_g*16, features_g*8, 4, 2, 1), # f_g*16 x f_g*8 x 8 x 8
            self._block(features_g*8, features_g*4, 4, 2, 1), # f_g*8 x f_g*4 x 16 x 16
            self._block(features_g*4, features_g*2, 4, 2, 1), #32 x 32
            nn.ConvTranspose2d(features_g*2, channels_img, 4, 2, 1),
            nn.Tanh() #[-1,1]
        )
    
    def _block(self, in_channels, out_channels, kernel_size, stride, padding):
        return nn.Sequential(
            nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU()
        )
    
    def forward(self,x):
        return self.net(x)

def initialize_weight(model):
     for m in model.modules():
            if isinstance(m,(nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
                nn.init.normal_(m.weight.data, 0.0, 0.02)
            
# def test():
#     N, in_channels, H, W = 8, 3, 64, 64
#     z_dim = 100
#     X = torch.randn((N, in_channels, H, W))
#     disc = Discriminator(in_channels,8)
#     assert disc(X).shape == (N, 1, 1, 1) # One Value per example
#     gen = Generator(z_dim, in_channels, 64)
#     z = torch.randn((N, z_dim, 1, 1))
#     assert gen(z).shape == (N, in_channels, H, W) # Ouput Generated image
#     print("Success")
# test()

In [6]:
## The training Setup
root = "C:\\Users\\shant\\celeba"
LEARNING_RATE = 2e-4
BATCH_SIZE = 128
IMAGE_SIZE = 64
# Image Channels in the generator output and input to the discriminator 
IMAGE_CHANNELS = 3

# Latent Space Dimensions
Z_DIM = 100
NUM_EPOCHS = 5

# feature size in the discriminator
FEATURES_DISC = 64

# feature size in the generator
FEATURES_GEN = 64

# Setup Transforms
dataset = datasets.ImageFolder(root=root, transform = transforms.Compose([
    transforms.Resize(IMAGE_SIZE),
    transforms.CenterCrop(IMAGE_SIZE),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
]))

In [7]:
# DataLoader
dataloader = DataLoader(dataset, batch_size = BATCH_SIZE, shuffle=True)

device  = torch.device("cuda" if torch.cuda.is_available() else "cpu")
generator = Generator(Z_DIM, IMAGE_CHANNELS, FEATURES_GEN).to(device)
discriminator = Discriminator(IMAGE_CHANNELS,FEATURES_DISC).to(device)
initialize_weight(generator)
initialize_weight(discriminator)

generator_optimizer = optim.Adam(generator.parameters(), lr = LEARNING_RATE, betas = (0.5,0.999))
discriminator_optimizer = optim.Adam(discriminator.parameters(), lr = LEARNING_RATE, betas = (0.5,0.999))

criterion = nn.BCELoss()

# Batch of Latent Vectors
fixed_noise = torch.randn((64, Z_DIM, 1, 1)).to(device)
writer_real = SummaryWriter(f"logs/real")
writer_fake = SummaryWriter(f"logs/fake")
step = 0 # Printing to tensorboard

generator.train()
discriminator.train()

for epoch in range(NUM_EPOCHS):
    
    # Unsupervised
    for batch_idx, (real, _) in enumerate(dataloader):
        # The real world images
        real = real.to(device)
        
        # Latent noise
        noise = torch.randn((BATCH_SIZE, Z_DIM, 1, 1)).to(device)
        
        # Pass the latent vector through the generator
        fake = generator(noise)
        
        # Train the discriminator max log(D(x)) + log(1-D(G(z)))
        # Flatten
        disc_real = discriminator(real).reshape(-1)
        
        # log(D(x)), y_n = ones hence only logx_n term is left. Refer to Pytorch Documentation
        loss_disc_real = criterion(disc_real, torch.ones_like(disc_real)) 
        disc_fake = discriminator(fake).reshape(-1)
        loss_disc_fake = criterion(disc_fake, torch.ones_like(disc_fake))
        
        # Addition of gradients from the all the real and fake samples
        loss_disc = (loss_disc_fake + loss_disc_real) / 2
        discriminator.zero_grad()
        loss_disc.backward(retain_graph = True) # Reutilize fake, but pytorch removes intermeditate results
        discriminator_optimizer.step()
        
        ## Train Generator max log(D(G(z)))
        output = discriminator(fake).reshape(-1)
        loss_gen = criterion(output, torch.ones_like(output))
        generator.zero_grad()
        loss_gen.backward()
        generator_optimizer.step()
        
        if batch_idx % 100 == 0:
            
            print(
            f"Epoch [{epoch}/{NUM_EPOCHS}] Batch {batch_idx}/{len(dataloader)} \
                  Loss D: {loss_disc:.4f}, loss G: {loss_gen:.4f}"
            )
            with torch.no_grad():
                fake = generator(fixed_noise)
            
                # The [:64] prints out the 4-D tensor BxCxHxW
                img_grid_real = torchvision.utils.make_grid(
                    real[:64], normalize = True)
                img_grid_fake = torchvision.utils.make_grid(
                    fake[:64], normalize = True)
                writer_real.add_image("Real", img_grid_real, global_step=step)
                writer_fake.add_image("Fake", img_grid_fake, global_step=step)
            step += 1 # See progression of images

Epoch [0/5] Batch 0/1583                   Loss D: 0.6823, loss G: 0.5775
Epoch [0/5] Batch 100/1583                   Loss D: 0.0121, loss G: 0.0088
Epoch [0/5] Batch 200/1583                   Loss D: 0.0029, loss G: 0.0022
Epoch [0/5] Batch 300/1583                   Loss D: 0.0012, loss G: 0.0009


KeyboardInterrupt: 